In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import bz2
import json
from tld import get_tld
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
import math
from collections import Counter
from operator import itemgetter
import re
import pickle

nltk.download()
nltk.download('stopwords')
nltk.download('punkt')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lisalaurent/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lisalaurent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from Data_clean_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lisalaurent/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lisalaurent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### CREATE PATHS: change according to local file

In [3]:
DATA_PATH = './data/'
FILE2016 = DATA_PATH + 'quotes-2016.json.bz2'
PATH_OUT = DATA_PATH + 'clean-quotes-2016.json.bz2'

In [4]:
df_base = pd.read_json(FILE2016, lines=True, compression='bz2', nrows=1000)
df_base

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
995,2016-02-18-024787,Everybody's watching the weather in India. The...,Mike Davey,"[Q6846495, Q6846496]",2016-02-18 10:00:48,3,"[[Mike Davey, 0.8103], [None, 0.1897]]",[http://www.cbc.ca/news/business/pulse-crops-l...,E
996,2016-12-19-016854,Everyone chipped in and did a great job.,None,[],2016-12-19 17:58:57,1,"[[None, 0.9884], [Tom Savage, 0.0116]]",[http://texashsfootball.com/east-blows-west-el...,E
997,2016-07-18-019424,"Everyone has a right to voice their opinion, e...",Norman Kwong,[Q119775],2016-07-18 14:05:38,1,"[[Norman Kwong, 0.7824], [None, 0.2176]]",[http://nbcnews.com/news/asian-america/asian-a...,E
998,2016-08-09-019117,Everyone has been so friendly and I cannot wai...,Guochuan Lai,[Q27975214],2016-08-09 17:14:51,4,"[[Guochuan Lai, 0.8836], [None, 0.1131], [Alan...",[http://expressandstar.com/sport/west-bromwich...,E


In [5]:
Wikidata_speakers = pd.read_parquet(DATA_PATH + 'speaker_attributes.parquet')
Wikidata_speakers

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[Q30],[Q6581097],1397399351,None,None,[Q82955],None,None,Q106406560,Barker B. Howard,None,item,None
9055977,[Charles Macomber],None,[Q30],[Q6581097],1397399471,None,None,[Q82955],None,None,Q106406571,Charles H. Macomber,None,item,None
9055978,None,[+1848-04-01T00:00:00Z],None,[Q6581072],1397399751,None,None,None,None,None,Q106406588,Dina David,None,item,None
9055979,None,[+1899-03-18T00:00:00Z],None,[Q6581072],1397399799,None,None,None,None,None,Q106406593,Irma Dexinger,None,item,None


In [6]:
Wikidata_countries = pd.read_csv(DATA_PATH + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression = 'bz2')
Wikidata_countries

,QID,Label,Description
0,Q31,Belgium,country in western Europe
1,Q45,Portugal,country in southwestern Europe
2,Q75,Internet,global system of connected computer networks
3,Q148,People's Republic of China,sovereign state in East Asia
4,Q155,Brazil,country in South America
...,...,...,...
36964,Q106302506,didgeridooist,musician who plays the didgeridoo
36965,Q106341153,biochemistry teacher,teacher of biochemistry at any level
36966,Q106368830,2018 Wigan Metropolitan Borough Council electi...,NaN
36967,Q106369692,2018 Wigan Metropolitan Borough Council electi...,NaN


## Create a dictionnary of categories and associated synonyms

In [7]:
matchers = {"art": ["art", "paint", "draw", "museum"], \
            "business": ["business", "finance", "economy", "commerce", "bank", "money", "trade"], \
            "entertainment":["entertainment"], 
            "fashion":["fashion", "couture", "designer"], \
            "medicine":["medicine", "health", "pharmacy", "wellbeing", "body"], \
            "music":["music", "song", "album", "concert"], \
            "politics":["politics", "government"], \
            "science":["science", "research"], \
            "sport": ["sport", "football", "athletics", "swimming", "rugby", "tennis", "volleyball", "ski"]}

In [ ]:
# Don't run this as doesn't return good list

'''

def find_synonyms(matchers):
    for category in matchers:
        synonyms = []
        for i in range(len(matchers[category])):
            word = matchers[category][i]
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    synonyms.append(l.name())
            matchers[category] = synonyms
    return matchers

matchers = find_synonyms(matchers)

'''

In [8]:
generalizeDictionary(matchers)

In [9]:
matchers

{'art': ['art', 'paint', 'draw', 'museum'],
 'business': ['busi',
  'financ',
  'economi',
  'commerc',
  'bank',
  'money',
  'trade'],
 'entertainment': ['entertain'],
 'fashion': ['fashion', 'coutur', 'design'],
 'medicine': ['medicin', 'health', 'pharmaci', 'wellb', 'bodi'],
 'music': ['music', 'song', 'album', 'concert'],
 'politics': ['polit', 'govern'],
 'science': ['scienc', 'research'],
 'sport': ['sport',
  'footbal',
  'athlet',
  'swim',
  'rugbi',
  'tenni',
  'volleybal',
  'ski']}

### PROCESSING DATA

In [13]:
# Pickle the chunks of dataframe for later
n_chunks = 0

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        n_chunks += 1
        chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers, Wikidata_speakers, Wikidata_countries)
        with open(PATH_OUT, 'ab') as d_file:
            pickle.dump(chunk_cleaned, d_file)       

Processing chunk with 1000 rows


KeyboardInterrupt: 

In [ ]:
# Open pickled file
for chunk_nbr in range(n_chunks):
    with open(PATH_OUT, 'rb') as d_file:
        chunk = pickle.load(d_file)
        # Need to load m times if you want to get to the m^th file

### Without pickle / writing to new json

In [10]:
with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    with bz2.open(PATH_OUT, 'wb') as d_file:
        for chunk in df_reader:
            chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers, Wikidata_speakers, Wikidata_countries)
            chunk_json = chunk_cleaned.to_json(orient='columns')#, index=False) #write it to pickle file instead
            d_file.write((chunk_json+'\n').encode('utf-8'))

Processing chunk with 1000 rows
Processing chunk with 1000 rows


KeyboardInterrupt: 

In [11]:
chunk_cleaned

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,gender,citizenship,Description,sitenames,domain,tags
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,Q2622812,2016-12-26 20:05:00,1,"[Andy Reid, 0.9432]",[http://www.kcchiefs.com/news/article-2/How-a-...,E,Male,United States of America,sovereign state in North America,[kcchiefs],[com],[[]]
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,Q6847325,2016-07-31 08:22:12,2,"[Mike Howe, 0.7118]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E,Male,United States of America,sovereign state in North America,"[peninsuladailynews, peninsuladailynews]","[com, com]","[[], []]"
2,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,Q6294,2016-07-11 17:26:06,1,"[Hillary Clinton, 0.9025]",[http://www.breitbart.com/tech/2016/07/11/hill...,E,Female,United States of America,sovereign state in North America,[breitbart],[com],[[]]
3,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,Q26129591,2016-05-26 15:21:37,1,"[Bruce Maxwell, 0.8178]",[http://www.scout.com/mlb/athletics/story/1673...,E,Male,NaN,NaN,[scout],[com],[[sport]]
4,2016-02-12-000118,[ It ] would give us more flexibility in terms...,Bob McDonald,Q41948618,2016-02-12 18:30:52,1,"[Bob McDonald, 0.9298]",[http://federalnewsradio.com/ses/2016/02/ses-m...,E,Male,NaN,NaN,[federalnewsradio],[com],[[]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,2016-12-11-010250,Everybody who lives in Lemont has been affecte...,Sue Smith,Q15040075,2016-12-11 10:00:00,1,"[Sue Smith, 0.857]",[http://www.statecollege.com/news/local-news/d...,E,Female,United Kingdom,country in Western Europe,[statecollege],[com],[[]]
566,2016-05-26-022023,Everybody's been supportive (but) everybody's ...,Mike Pennel,Q17712918,2016-05-26 20:55:46,1,"[Mike Pennel, 0.551]",[http://www.scout.com/nfl/packers/story/167313...,E,Male,United States of America,sovereign state in North America,[scout],[com],[[]]
567,2016-07-18-019424,"Everyone has a right to voice their opinion, e...",Norman Kwong,Q119775,2016-07-18 14:05:38,1,"[Norman Kwong, 0.7824]",[http://nbcnews.com/news/asian-america/asian-a...,E,Male,Canada,sovereign state in North America,[nbcnews],[com],[[]]
568,2016-08-09-019117,Everyone has been so friendly and I cannot wai...,Guochuan Lai,Q27975214,2016-08-09 17:14:51,4,"[Guochuan Lai, 0.8836]",[http://expressandstar.com/sport/west-bromwich...,E,Male,People's Republic of China,sovereign state in East Asia,"[expressandstar, standard, standard, standard]","[com, co.uk, co.uk, co.uk]","[[sport], [sport, sport], [sport, sport], [spo..."


In [ ]:
df = pd.read_json(PATH_OUT, orient='columns', compression='bz2')
df

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2', lines=True, nrows=100)
df

In [ ]:
with pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    
